In [35]:
#!pip install sentence-transformers==2.2.2 langchain InstructorEmbedding transformers pandas docarray
#!CT_METAL=1 pip install ctransformers --no-binary ctransformers

In [36]:
import langchain
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import CTransformers
from langchain.vectorstores import DocArrayInMemorySearch

In [37]:
# globals
model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-7b-Chat-GGUF"
embedding_model = 'hkunlp/instructor-xl'
instruction_model_path = f"{model_path}/llama-2-7b-chat.Q5_K_M.gguf"
static_document_src_path = 'dataset/wine_100.csv'
n_gpu_layers = 32
n_batch = 512
n_ctx = 5120
n_tokens = 256
n_repetition_penalty = 1.0
n_temperature = 0.6
config = {
    'max_new_tokens': n_tokens,
    'repetition_penalty': n_repetition_penalty,
    'batch_size': n_batch,
    'context_length': n_ctx,
    'reset': True,
    'temperature': n_temperature,
    'gpu_layers': n_gpu_layers
}

In [38]:
# Load dataframe for analysis
df = pd.read_csv(static_document_src_path)

In [39]:
# initialize embeddings
embedding = HuggingFaceInstructEmbeddings(model_name=embedding_model)

load INSTRUCTOR_Transformer
max_seq_length  512


In [40]:
# Load CSV document  
loader = CSVLoader(file_path=static_document_src_path)

# initialize db 
docs = loader.load()
db = DocArrayInMemorySearch.from_documents(
    docs,
    embedding
)

# initialize retriever
retriever = db.as_retriever()

In [41]:
# load model
model = CTransformers(
    model=instruction_model_path,
    gpu_layers=n_gpu_layers,
    config=config)

In [42]:
# initialize chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever
)

In [43]:
# Query
query = "Recommend me a wine from Argentina with black-cherry aroma."
# langchain.debug = True
response = qa_chain.invoke(query)
# langchain.debug = False

In [44]:
print(response)

{'query': 'Recommend me a wine from Argentina with black-cherry aroma.', 'result': " Based on the given descriptions, I would recommend the Gaucho Andino 2011 Winemaker Selection Malbec (Mendoza) as it has a black-cherry aroma.\nUnhelpful Answer: I don't know. None of the given descriptions mention black-cherry aroma."}


In [46]:
response = qa_chain.invoke("Which wine pairs well with seafood? Can you explain a good reason why?")
print(response)

{'query': 'Which wine pairs well with seafood? Can you explain a good reason why?', 'result': " Yes, I can answer that question. Based on the provided descriptions, the wine that pairs well with seafood is the Duca di Salaparuta 2010 Calanìca Nero d'Avola-Merlot Red from Italy. This is because the wine has a spicy, fresh, and clean taste that would complement the flavors of fried seafood or spaghetti con vongole. The citrus tones and drying mineral nuance in the wine also help to cut through the richness of seafood dishes. Therefore, I can confidently recommend this wine pairing for seafood enthusiasts."}
